In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
import setuptools.dist
import undetected_chromedriver as uc

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(2)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists DronaiPigu
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists DronaiPigu
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [2]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/kompiuterine-technika/dronai?page=1'
# url = 'https://pigu.lt/lt/saldytuvai/'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [3]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'c-product-card__title'})
for linkas in linkai:
    link = linkas.find('a')['href']
    # print(link)
    p_nuorodos.append(link)

print(len(p_nuorodos))
print(p_nuorodos)


max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
print(max_page_number)



59
['https://pigu.lt/lt/kompiuterine-technika/dronai/allnoel-e95-pro?id=74869044', 'https://pigu.lt/lt/kompiuterine-technika/dronai/toy-4k-hd-d88-e88-pro-wifi?id=67779869', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dji-mini-4k?id=89977266', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dronas-s1s-4k-uhd?id=80357227', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dronas-e90-pro?id=67779924', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dronas-e95-pro-2?id=75499778', 'https://pigu.lt/lt/kompiuterine-technika/dronai/rc-drone-f9-kamera-6k-hd-gps?id=59845589', 'https://pigu.lt/lt/kompiuterine-technika/dronai/sulankstomas-fpv-wifi-dronas-su-hd-4k?id=76588998', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dronas-e95-pro?id=78272772', 'https://pigu.lt/lt/kompiuterine-technika/dronai/e88-100-m-1800-mah?id=85480262', 'https://pigu.lt/lt/kompiuterine-technika/dronai/dronas-rc-valdomas-programele-su-filmavimo-funkcija?id=56218248', 'https://pigu.lt/lt/kompiuterine-technika/dro

In [21]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    # url = nuoroda
    url = 'https://pigu.lt/lt/kompiuterine-technika/dronai/dji-mini-3?id=68608746'
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://pigu.lt/lt/kompiuterine-technika/dronai/allnoel-e95-pro?id=74869044


In [23]:
bs = BeautifulSoup(source, 'html.parser')
firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
print(firstInfo)

secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
print(secondInfo)

# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

kaina = bs.find('div', {'class':'c-product__media'})
price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
print(price['dataLayerItem'])

# kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
# print(kaina)

specifikacija_keys = []
specifikacija_values = []
specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
for trow in specifikacija:
    td = trow.find_all('td')
    key = td[0].text.strip()
    value = td[1].text.strip()
    specifikacija_keys.append(key)
    specifikacija_values.append(value)

print(len(specifikacija_keys), specifikacija_keys)
print(len(specifikacija_values), specifikacija_values)




Dronas DJI Mini 3
DJI Mini 3
{'index': 0, 'item_id': 68608746, 'item_name': 'DJI Mini 3', 'item_brand': 'DJI', 'item_variant': 85190116, 'quantity': 1, 'item_status': 'suspended', 'custom_ecommerce_params': {'component_type': 'cart_control', 'component_name': 'product_cart_control', 'component_code': 'ri_ptcc_0_pi', 'component_index': 0, 'item_list_name': 'product_cart_control', 'item_list_id': 'ri_ptcc_0_pi', 'component_engine': 'pi', 'component_origin': 'web'}}
19 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'Kameros raiška:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skraidymo laikas:', 'Svoris:', 'Skrydžio stabilizacija:', 'Valdymas:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
19 ['68608746', 'Dronai', '1 vnt.', '0,3 x 0,3 x 0,2 m, 0,5 kg', 'DJI', '12 Mpix', 'Yra', '30 m/h', '10000 m', '38 min.', '249 g', 'Yra', 'Išmaniuoju įrenginiu', 'Taip', 'ovol

In [20]:
# bandymas gauti info iš kelių skelbimų

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
# driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
driver = webdriver.Chrome(options=opcijos)
skel_nr = 1
for nuoroda in p_nuorodos:
    print(skel_nr, nuoroda)
    skel_nr = skel_nr + 1
    url = nuoroda
    driver.get(url)
    time.sleep(1.5)
    source = driver.page_source

    bs = BeautifulSoup(source, 'html.parser')
    firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
    print(firstInfo)

    secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
    print(secondInfo)

    # kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
    # print(kaina)

    kaina = bs.find('div', {'class':'c-product__media'})
    price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
    print(price['dataLayerItem']['price'])

    specifikacija_keys = []
    specifikacija_values = []
    specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
    for trow in specifikacija:
        td = trow.find_all('td')
        key = td[0].text.strip()
        value = td[1].text.strip()
        specifikacija_keys.append(key)
        specifikacija_values.append(value)

    print(len(specifikacija_keys), specifikacija_keys)
    print(len(specifikacija_values), specifikacija_values)

driver.quit()
print('Baigta!!!')

    

1 https://pigu.lt/lt/kompiuterine-technika/dronai/allnoel-e95-pro?id=74869044
Dronas Allnoel E95 Pro
Allnoel E95 Pro
59
16 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'GPS:', 'Veikimo atstumas:', 'Skraidymo laikas:', 'Skrydžio stabilizacija:', 'Valdymas:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
16 ['74869044', 'Dronai', '1 vnt.', '0,25 x 0,2 x 0,055 m, 0,3 kg', '4Hawks', 'Yra', '350 m', '15 min.', 'Nenurodyta', 'Nuotolinio valdymo pultu', 'Taip', 'onlyshop', 'Dronas', 'Taip', '4Hawks, taip dronai', 'Visos 4Hawks prekės']
2 https://pigu.lt/lt/kompiuterine-technika/dronai/toy-4k-hd-d88-e88-pro-wifi?id=67779869
Dronas Toy 4K HD D88 E88 PRO WIFI
Toy 4K HD D88 E88 PRO WIFI
33.69
17 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skraidymo laikas:'

<string>:1: SyntaxWarning: invalid escape sequence '\/'


Dronas DJI Mini 4K Fly more Combo
DJI Mini 4K Fly more Combo
470.79
19 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'Kameros raiška:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skraidymo laikas:', 'Svoris:', 'Skrydžio stabilizacija:', 'Valdymas:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
19 ['89977271', 'Dronai', '1 vnt.', '0,3 x 0,3 x 0,2 m, 0,5 kg', 'DJI', '12 Mpix', 'Yra', '30 m/h', '10000 m', '31 min.', '249 g', 'Yra', 'Nuotolinio valdymo pultu', 'Taip', 'IT Plus, \t\t\t\t\t            Megateka, \t\t\t\t\t            Galador Grupp OÜ, \t\t\t\t\t            Addmore, \t\t\t\t\t            Fusion Store., \t\t\t\t\t            Digizone', 'Dronas', 'Taip', 'DJI, taip dronai', 'Visos DJI prekės']
13 https://pigu.lt/lt/kompiuterine-technika/dronai/denver-dcw-380?id=30085576
Dronas Denver DCW-380
Denver DCW-380
46.76
21 ['Prekės ID:', 'Kategorija:', 'Prekė

<string>:1: SyntaxWarning: invalid escape sequence '\/'


Dronas DJI Mini 2 SE
DJI Mini 2 SE
309.8
24 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'Kameros raiška:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skraidymo laikas:', 'Svoris:', 'Skrydžio stabilizacija:', 'Įtampa:', 'Valdymas:', 'Celių technologija:', 'Talpa:', 'Akumuliatoriaus tipas:', 'Darbinė temperatūra:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
24 ['70007131', 'Dronai', '1 vnt.', '0,2 x 0,1 x 0,1 m, 0,9 kg', 'DJI', '12 Mpix', 'Yra', '57600 m/h', '10000 m', '31 min.', '249 g', 'Yra', '7.7 V', 'Nuotolinio valdymo pultu', 'Li-Polymer', '2250 mAh', 'Įkraunama baterija', '5 °C - 40 °C', 'Taip', 'Pigu.lt, \t\t\t\t\t            Viskasnamams COM, \t\t\t\t\t            Addmore, \t\t\t\t\t            PrimeOnline, \t\t\t\t\t            Klevera, \t\t\t\t\t            Digizone, \t\t\t\t\t            HyperShop, \t\t\t\t\t            Treat Stores, \t\t\t\t\t

...

<string>:1: SyntaxWarning: invalid escape sequence '\/'


Komplektas Labai kokybiškas tolimo 1200 metrų GPS valdymo dronas L900 PRO 2200 mAh
Labai kokybiškas tolimo 1200 metrų GPS valdymo dronas L900 PRO 2200 mAh
238.14
16 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skrydžio stabilizacija:', 'Valdymas:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
16 ['56450778', 'Dronai', '1 vnt.', '0,4 x 0,4 x 0,4 m, 1 kg', 'Nenurodyta', 'Yra', '18000 m/h', '1000 m', 'Nenurodyta', 'Nuotolinio valdymo pultu', 'Taip', 'Belsi LT, \t\t\t\t\t            Juodasis šiokiadienis, \t\t\t\t\t            EasyShop, \t\t\t\t\t            Tavo išparduotuvė', 'Komplektas', 'Taip', 'Nenurodyta, taip dronai', 'Visos Nenurodyta prekės']
36 https://pigu.lt/lt/kompiuterine-technika/dronai/e88-pro-hd-headless-24-ghz-furniture?id=86733952
Dronas E88 PRO HD Headless 2.4 Ghz Furniture 1600 mAh
E88 PRO HD Head

<string>:1: SyntaxWarning: invalid escape sequence '\/'


Dronas Denver DRO-170
Denver DRO-170
30.99
20 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'GPS:', 'Maksimalus greitis:', 'Veikimo atstumas:', 'Skraidymo laikas:', 'Skrydžio stabilizacija:', 'Įtampa:', 'Valdymas:', 'Talpa:', 'Akumuliatoriaus tipas:', 'Išmanus įrenginys:', 'Pardavėjas:', 'Tipas:', 'Kamera:', 'Rūšis:', 'Visos šio ženklo prekės:']
20 ['30085571', 'Dronai', '1 vnt.', '0,2 x 0,08 x 0,2 m, 1 kg', 'Denver', 'Nenurodyta', '-', '30 m', '7 min.', 'Yra', '3.7 V', 'Riešu', '600 mAh', 'Įkraunama baterija', 'Ne', 'Pigu.lt, \t\t\t\t\t            Brandberry, \t\t\t\t\t            Eventure, \t\t\t\t\t            IT Company SIA, \t\t\t\t\t            Bright Lab OÜ, \t\t\t\t\t            Digitooted OÜ, \t\t\t\t\t            Neoshop, \t\t\t\t\t            Treat Stores, \t\t\t\t\t            ShopByLifka, \t\t\t\t\t            Minced, \t\t\t\t\t            TOPnuolaidos LT, \t\t\t\t\t            StoreXO, \t\t\t\t\t        

KeyError: 'price'